In [58]:
import pickle
import numpy

In [67]:
[user_rev_sample_dict,item_rev_sample_dict] = pickle.load(open("user_item_rev_samples.pkl","rb"))

In [68]:
def create_epoch(inp_path,out_path):
    fo = open(out_path,"w")
    k=0
    with open(inp_path) as f:
        try:
            for line in f:
                if k%10000==0:
                    print k,
                user_id,item_id,rating,review_id = line.strip().split("\t")
                user_vec_t = user_rev_sample_dict[user_id]
                item_vec_t = item_rev_sample_dict[item_id]
                threshold = len(user_vec_t) -1
                user_vec,item_vec = [],[]
                for i in range(threshold+1):
                    if len(user_vec) < threshold and user_vec_t[i][0] != review_id:
                        user_vec.append(rp.get_emb(user_vec_t[i][0]))
                    if len(item_vec) < threshold and item_vec_t[i][0] != review_id:
                        item_vec.append(rp.get_emb(item_vec_t[i][0]))
                user_vec =numpy.array(user_vec).flatten().tolist()
                item_vec= numpy.array(item_vec).flatten().tolist()
                fo.write("\t".join([user_id,item_id,rating," ".join([str(x) for x in user_vec]),\
                                    " ".join([str(x) for x in item_vec])])+"\n")
                k+=1
        except Exception:
            print item_vec_t,item_id,len(item_vec_t)

In [ ]:
create_epoch('train_review_data.txt','train_epoch_data.txt')

0 10000 20000 30000 40000

In [4]:
class ReviewProcessing:
    def __init__(self):
        self.review_dict = {}
        
    def load(self,review_delim_path,review_emb_path):
        self.review_dict = pickle.load(open(review_delim_path,"rb"))
        self.review_emb_path = review_emb_path
    
    def get_emb(self,review_id):
        if not self.review_dict:
            raise Exception("load the word index and emb files")
        if not review_id in self.review_dict:
            return None
        start,length = self.review_dict[review_id]
        with open(self.review_emb_path,"r") as f:
            f.seek(start)
            line = f.read(length)
            emb = line.split("\t")[4]
            emb = numpy.array([float(x) for x in emb.split()])
        return emb
    

In [5]:
rp = ReviewProcessing()
rp.load("review_delim.pkl","review_emb.txt")

In [51]:
# f=open('train_epoch_data.txt','r')
# for l in f:
#     u,i,r,u_e,i_e=l.split("\t")
#     print u,i
#     u_e = numpy.array([float(x) for x in u_e.split(" ")]).reshape(10,64)
#     print u_e